In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
from collections import defaultdict
import pickle

In [2]:
def get_page_content(request_url):
    headers={'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.104 Safari/537.36'}
    try:
        r = requests.get(request_url, headers=headers,timeout=10)
    except requests.exceptions.SSLError as err:
        print(err)
        r = requests.get(request_url, headers=headers,timeout=10, verify=False)
    content=r.text
    soup=BeautifulSoup(content,'html.parser',from_encoding='utf-8')
    return soup

In [3]:
request_url='https://ditie.mapbar.com/shanghai_line/'
soup=get_page_content(request_url)

d:\program files\python\lib\site-packages\bs4\__init__.py:221: UserWarning: You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.
  warnings.warn("You provided Unicode markup but also provided a value for from_encoding. Your from_encoding will be ignored.")


In [4]:
subways=soup.find_all('div',attrs={'class':'station'})
df=pd.DataFrame(columns=['route_name','station'])
for subway in subways:
    route_name=subway.find('strong',attrs={'class':'bolder'}).text
    routes=subway.find('ul')
    stations=routes.find_all('a')
    for station in stations:
        temp={'station':station.text,'route_name':route_name}
        df=df.append(temp,ignore_index=True)
df['city']='上海'
##df.to_excel('./shanghai_subway.xlsx',index=False)
        

In [5]:
def get_location(keywords,city):
    headers={'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.104 Safari/537.36'}
    request_url='http://restapi.amap.com/v3/place/text?key=d5104f60b63fdcb83705a6f14e155145&keywords='+keywords+'&types=&city='+city+'&children=1&offset=1&page=1&extensions=all'
    try:
        r = requests.get(request_url, headers=headers,timeout=10)
    except requests.exceptions.SSLError as err:
        print(err)
        r = requests.get(request_url, headers=headers,timeout=10, verify=False)
    r.encoding='utf-8'
    data=r.text
    pattern='location":"(.*?),(.*?)"'
    result=re.findall(pattern,data)
    #print(result)
    return result[0][0],result[0][1]

In [6]:
df['longitude'],df['latitude']=None,None
for index,row in df.iterrows():
    longitude,latitude=get_location(row['station'],row['city'])
    df.iloc[index]['longitude']=longitude
    df.iloc[index]['latitude']=latitude
df.to_excel('./shanghai_subway.xlsx',index=False)

In [7]:
#利用高德地图提供的API计算并获取每两地铁站之间的距离和所需时间
def compute_distance_duration(log1,lat1,log2,lat2):
    headers={'user-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.104 Safari/537.36'}
    request_url='http://restapi.amap.com/v3/distance?key=d5104f60b63fdcb83705a6f14e155145&origins='+str(log1)+','+str(lat1)+'&destination='+str(log2)+','+str(lat2)+'&type=1'
    try:
        r = requests.get(request_url, headers=headers,timeout=10)
    except requests.exceptions.SSLError as err:
        print(err)
        r = requests.get(request_url, headers=headers,timeout=10, verify=False)
        r.encoding='utf-8'
    data=r.text
    pattern='distance":"(.*?)","duration":"(.*?)"'
    result=re.findall(pattern,data)
    return result[0][0],result[0][1]

In [8]:
#建立每一个station与它相邻的station之间的距离和时间关系表
def get_graph():
    graph_distance,graph_duration=defaultdict(dict),defaultdict(dict)
    for i in range(df.shape[0]):
        route1=df.iloc[i]['route_name']
        if i <df.shape[0]-1:
            route2=df.iloc[i+1]['route_name']
            if route1==route2:
                longtitude1,latitude1=df.iloc[i]['longitude'],df.iloc[i]['latitude']
                longtitude2,latitude2=df.iloc[i+1]['longitude'],df.iloc[i+1]['latitude']
            
                station1=df.iloc[i]['station']
                station2=df.iloc[i+1]['station']
            
                distance,duration=compute_distance_duration(longtitude1,latitude1,longtitude2,latitude2)
                graph_distance[station1][station2]=distance
                graph_distance[station2][station1]=distance
            
                graph_duration[station1][station2]=duration
                graph_duration[station2][station1]=duration
    return graph_distance,graph_duration

In [9]:
#将建立的station之间的距离和时间关系表保存，方便调用
graph_distance,graph_duration=get_graph()
output_graph_distance=open('graph_distance.pkl','wb')
pickle.dump(graph_distance,output_graph_distance)
output_graph_duration=open('graph_duration.pkl','wb')
pickle.dump(graph_duration,output_graph_duration)

In [22]:
#寻找最低开销节点
def find_lowest_cost_node(costs,processed):
    #print('processed:',processed)
    #定义初始的最低开销为无穷大
    lowest_cost=float('inf')
    lowest_cost_node=None
    #遍历已经进入costs中的每一个节点，并获取每一个节点开销，并与lowest_cost进行比较和更新
    for node in costs:
        #已经完成并加入processed中的节点不再遍历
        if not node in processed:
            if costs[node]<lowest_cost:
                lowest_cost=costs[node]
                lowest_cost_node=node
    return lowest_cost_node

In [23]:
#根据起始站和终点站返回最优路径
def find_shortest_path(start_station,end_station,parents):
    node=end_station
    shortest_path=[end_station]
    #从后往前推路径，最后的根节点为start
    #print(start_station)
    #循环，直至找到起始站
    while parents[node]!=start_station:
        shortest_path.append(parents[node])
        node=parents[node]
    shortest_path.append(start_station)
    return shortest_path

In [44]:
#定义Dijkstra方法，并利用该方法寻找每一个节点至start节点的最低开销关系
def dijkstra(costs,processed,graph,parents):
    #获取到start的最小开销节点node
    node=find_lowest_cost_node(costs,processed)
    while node is not None:
        #获取当前node的开销值（即当前node到start的开销值）
        cost_node_start=costs[node]
        #print('costs:',costs)
        #print('{}-{}'.format(node,costs[node]))
        #获取当前node的邻居
        for neighber in graph[node].keys():
            if not neighber in processed:
                #print(neighber)
                #计算每一个邻居到当前node的开销
                cost_neighber_node=float(graph[node][neighber])
                #每一个邻居到start的开销值=每一个邻居到当前node的开销+当前node到start的开销
                cost_neighber_start=cost_neighber_node+cost_node_start
                #如果新求得的neighber到start开销值比已知的开销值小，则将开销值costs[neighber]进行更新
                if neighber not in costs or cost_neighber_start<costs[neighber]:
                    costs[neighber]=cost_neighber_start
                    #记录当前neighber的父节点
                    parents[neighber]=node
                    #将当前节点标记为已经处理,相当于Dijkstra算法中的S集合
                    #print(parents)
        processed.append(node)
        #print(processed)
        #继续寻找U中最小开销节点，然后对node进行更新，再次循环。costs相当于Dijkstra算法中的集合U
        node=find_lowest_cost_node(costs,processed)
        #print('当前cost最小节点为{}，cost值为{}'.format(node,costs[node])) 

In [45]:
#建立上海地铁的查询方法，输入参数为：起始站，终点站，以及按距离或时间方法检索。graph_index确定按时间最短还是距离最短，‘1’表示时间，‘0’表示距离
def metro_navigation_shanghai(start_station,end_station,graph_index):
    if graph_index=='0':
        file=open('graph_distance.pkl','rb')
    else:
        file=open('graph_duration.pkl','rb')
           
    graph=pickle.load(file)
    
    #创建节点的开销表，costs是指从start到该节点的距离
    costs={}
    #存储父节点，用于记录路径
    parents={}
    parents[end_station]=None
    #记录处理过的节点list
    processed=[start_station]
    
    costs[end_station]=float('inf')
    
    #从起始节点开始，计算起始节点相邻节点的开销值，并将起始节点标记为这些相邻节点的父节点
    for node in graph[start_station].keys():
        costs[node]=float(graph[start_station][node])
        parents[node]=start_station

    dijkstra(costs,processed,graph,parents)
    #获取最短路径
    shortest_path=find_shortest_path(start_station,end_station,parents)
    best_path='END'
    for paths in shortest_path:
        best_path=paths+'->'+best_path
    return best_path  

In [46]:
#按距离最短
start_station='虹口足球场站'
end_station='临平路站'
shortest_path=metro_navigation_shanghai(start_station,end_station,'0')
print('{}至{}距离最短最佳地铁路线是:{}'.format(start_station,end_station,shortest_path))

虹口足球场站至临平路站距离最短最佳地铁路线是:虹口足球场站->东宝兴路站->宝山路站->海伦路站->临平路站->END


In [31]:
#按时间最短
start_station='莘庄站'
end_station='紫藤路站'
shortest_path=metro_navigation_shanghai(start_station,end_station,'1')
print('{}至{}时间最短最佳地铁路线是:{}'.format(start_station,end_station,shortest_path))

莘庄站至紫藤路站时间最短最佳地铁路线是:莘庄站->外环路站->莲花路站->锦江乐园站->上海南站->漕宝路站->上海体育馆站->宜山路站->虹桥路站->宋园路站->伊犁路站->水城路站->龙溪路站->龙柏新村站->紫藤路站->END
